In [5]:
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

# Load image data


def load_images(directory):
    images = []
    labels = []
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
            img = cv2.imread(os.path.join(
                directory, filename), cv2.IMREAD_GRAYSCALE)
            if img is not None:
                img = cv2.resize(img, (300, 400))  # Resize image to (300, 400)
                images.append(img)
                # Extract label from the entire file name
                # Extracts the filename without extension
                label = os.path.splitext(filename)[0]
                labels.append(label)
    # Return list of original file names as labels
    return np.array(images), labels


# Define the base directory where the face dataset is located
base_directory = "Face_Output"
train_dir = "Face_Cropped_Split/Train"
test_dir = "Face_Cropped_Split/Test"  # Adjusted test directory

print("Number of training images:", len(
    os.listdir(os.path.join(base_directory, train_dir))))
print("Number of test images:", len(
    os.listdir(os.path.join(base_directory, test_dir))))

# Load training data
X_train, y_train = load_images(os.path.join(base_directory, train_dir))

# Check the number of unique classes in the training data
num_classes = len(np.unique(y_train))

# Normalize pixel values to range [0, 1]
X_train = X_train.astype('float32') / 255.0

# Reshape input data to match model input shape
X_train = np.expand_dims(X_train, axis=-1)  # Add a single channel dimension

# Define the CNN architecture for facial recognition
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(400, 300, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    # Output layer with softmax activation
    # Ensure num_classes matches the number of unique labels
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Load test data
X_test, y_test = load_images(os.path.join(base_directory, test_dir))

# Normalize pixel values to range [0, 1]
X_test = X_test.astype('float32') / 255.0

# Reshape test data to match model input shape
X_test = np.expand_dims(X_test, axis=-1)  # Add a single channel dimension

# Train the model
model.fit(X_train, np.arange(len(y_train)), epochs=5, validation_data=(
    X_test, np.arange(len(y_test))))  # Pass numerical labels

# Evaluate the model
test_loss, test_acc = model.evaluate(
    X_test, np.arange(len(y_test)))  # Pass numerical labels

# Convert accuracy to percentage
test_acc_percentage = test_acc * 100

print("Test Accuracy: {:.2f}%".format(test_acc_percentage))

# Predict labels for test images and print filenames along with predicted labels
print("\nImage File Name | Predicted Label | Match")
print("-------------------------------------------")
for i in range(len(X_test)):
    # Get the filename of the current image
    filename = os.listdir(os.path.join(base_directory, test_dir))[i]
    # Extract the first 5 characters from the filename
    filename_prefix = filename[:5]
    # Predict the label for the current image
    predicted_label = np.argmax(model.predict(
        np.expand_dims(X_test[i], axis=0)))
    # Check if the predicted label matches the first 5 characters of the filename
    match = "Yes" if str(predicted_label) == filename_prefix else "No"
    # Print the filename, predicted label, and whether there's a match
    print("{:<15} | {:<15} | {:<5}".format(filename, predicted_label, match))

Number of training images: 380
Number of test images: 95
Epoch 1/5
12/12 [==============================] - 22s 2s/step - loss: 6.0488 - accuracy: 0.0000e+00 - val_loss: 5.9422 - val_accuracy: 0.0105
Epoch 2/5
12/12 [==============================] - 20s 2s/step - loss: 5.9429 - accuracy: 0.0000e+00 - val_loss: 5.9419 - val_accuracy: 0.0000e+00
Epoch 3/5
12/12 [==============================] - 20s 2s/step - loss: 5.9342 - accuracy: 0.0026 - val_loss: 5.9501 - val_accuracy: 0.0000e+00
Epoch 4/5
12/12 [==============================] - 20s 2s/step - loss: 5.8961 - accuracy: 0.0053 - val_loss: 5.9302 - val_accuracy: 0.0316
Epoch 5/5
3/3 [==============================] - 1s 260ms/step - loss: 6.0054 - accuracy: 0.0105
Test Accuracy: 1.05%

Image File Name | Predicted Label | Match
-------------------------------------------
1/1 [==============================] - 0s 101ms/step
S0001_F_30_1.jpg | 250             | No   
1/1 [==============================] - 0s 45ms/step
S0001_F_30_10.jpg 